In [60]:
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from zipfile import ZipFile

In [61]:
corpus = []


archive = ZipFile('corpus2mw.zip', 'r')

fileList = archive.namelist()
for file in fileList:
    corpus.append((archive.read(file)).decode('UTF-8'))


In [62]:
regexp = re.compile('[a-zA-Z0-9 éáíúóãõçôâ-]') 

In [63]:
def processText(corpus):
    corp = []
    for text in corpus:
        listT = list(text)
        i = 0
        for c in listT:
            if not regexp.search(c) and not listT[i-1]==' ':
                listT.insert(i, ' ')
            i +=1
        corp.append(''.join(listT))
    return corp

corpus =  processText(corpus)

In [64]:
def dice(freq, pref_freqs, suff_freqs):
    pref_freqs = list(pref_freqs)
    suff_freqs = list(suff_freqs)
    return 2 * freq / (sum(pref_freqs) / len(pref_freqs) + sum(suff_freqs) / len(suff_freqs))


def scp(freq, pref_freqs, suff_freqs):
    multiplied_freqs = [pref_freq * suff_freq for pref_freq, suff_freq in zip(pref_freqs, suff_freqs)]
    return freq ** 2 * len(multiplied_freqs) / sum(multiplied_freqs)

In [65]:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(2, 7), token_pattern = '[a-zA-Z0-9$&+,:;=?@#|<>.^*()%!-]+')
vec_fit = vectorizer.fit_transform(corpus)
word_list = vectorizer.get_feature_names()
count_list = np.asarray(vec_fit.sum(axis=0))[0]
freq_dict = dict(zip(word_list,count_list))

In [149]:
filtered_dict = {k: v for k, v in freq_dict.items() if v > 1}

In [231]:
relev_expressions={ }


for key, value in filtered_dict.items():
    n= len(key.split(' '))
    if n>3:
        own_freq = value/(len(corpus)-n)
        #print(key)
        ownpref=''
        ownsuf=''
        for i in range(0,n):
            if i==0:
                ownpref += key.split(' ')[i]
            elif i==n-1:
                ownsuf += key.split(' ')[i]
            else:
                ownpref += ' ' + key.split(' ')[i]
                ownsuf += key.split(' ')[i] + ' '
        #print(ownpref)
        #print(ownsuf)
        own_pref_freq= filtered_dict[ownpref]/(len(corpus)-(n-1))
        own_suff_freq= filtered_dict[ownsuf]/(len(corpus)-(n-1))
        gluew= own_freq**2 /(own_pref_freq * own_suff_freq)
        #print(gluew)

        if own_pref_freq>own_suff_freq:
            aux= ownpref
            auxfreqy= own_pref_freq
        else:
            aux= ownsuf
            auxfreqy= own_suff_freq
            
        prefs=''
        sufs=''
        for i in aux.split(' ')[:-1]:
            prefs+=i+' '
        for i in aux.split(' ')[1:]:
            sufs+=i+' '
        
        own_pref_freq= filtered_dict[prefs.strip()]/(len(corpus)-(n-2))
        own_suff_freq= filtered_dict[sufs.strip()]/(len(corpus)-(n-2)) 
       
        gluex= auxfreqy**2 /(own_pref_freq * own_suff_freq)
        #print('x', gluex)
        
        besty=''
        bestfreqy=0
        for cha in filtered_dict.keys():
            if key.split(' ')[:]==(cha.split(' ')[:-1] or cha.split(' ')[1:]):
                #print(key, cha)
                wy= filtered_dict[cha]/(len(corpus)-(n+1))
                if wy>bestfreqy:
                    bestfreqy=wy
                    besty=cha
                          
        if(bestfreqy>0):
            pref=''
            suf=''
            for i in besty.split(' ')[:-1]:
                pref+=i+' '
            for i in besty.split(' ')[1:]:
                suf+=i+' '  
                        
            own_pref_freq= filtered_dict[pref.strip()]/(len(corpus)-(n-2))
            own_suff_freq= filtered_dict[suf.strip()]/(len(corpus)-(n-2)) 
        
            gluey= bestfreqy**2 /(own_pref_freq * own_suff_freq)
            
            re= (gluex+gluey)/2
            #print('re', re)
            #print('gluew', gluew)
            if re<gluew:
                relev_expressions[key]=re
                print(key)
        else:
            continue
    
        
        
        
    

00 with high score
000 copies in its first
000 from the kitty
000 girls aged 15
000 passengers per weekday
10 14 occupation of
10 day contract with the
10 day contract with the milwaukee
10 in big south
10 states in average
10 who were 65
10 years 2000 2010
10 years the population
10 years the population has
100 colony forming units of
100 females age 18
100 females age 18 and
100 females there were
100 females there were 100 males
100 females there were 103 males
100 females there were 98 males
100 man department of
100 man department of state
100 to reviews from
13 of the population
14 businesses in this
14 occupation of raleigh
15 african american 25
15 and the average
173 and the median
17th in the ap
18 11 from 18
18 11 from 18 to
18 90 from 18
18 90 from 18 to
18 and 11 of
18 and 11 of those
18 and of those
18 and of those age 65
18 and over there
18 and over there were
18 from 18 to 24
18 living with them
18 living with them 50 were
18 living with them 53 were
18 to 24 24 from
1

In [ ]:

vectorizer = CountVectorizer(analyzer='word', token_pattern = '[a-zA-Z0-9$&+,:;=?@#|<>.^*()%!-]+')
vec_fit = vectorizer.fit_transform(corpus)
single_word_list = vectorizer.get_feature_names_out()
single_count_list = np.asarray(vec_fit.sum(axis=0))[0]
single_freq_dict = dict(zip(single_word_list,single_count_list))

single_freq_dict = {key:val for key, val in single_freq_dict.items() if val > 1}
single_freq_dict = {k: v for k, v in sorted(single_freq_dict.items(), key=lambda item: item[1])}
values = np.fromiter(single_freq_dict.values(), dtype=float)
stop_words_list = np.stack((np.arange(0, len(single_freq_dict)), values), axis = -1)
list_of_counts = list(single_freq_dict.items())


from kneebow.rotor import Rotor
 
rotor = Rotor()
rotor.fit_rotate(stop_words_list)
elbow_idx = rotor.get_elbow_index()
print(elbow_idx)  
#rotor.plot_elbow()


'''
from kneed import KneeLocator
kn = KneeLocator(stop_words_list[:,0] ,stop_words_list[:,1], curve='convex', direction='increasing')
print(int(kn.knee))

stop = 0
deltaX = 100
for idx, i, j in zip(range(0, len(values)), values, values[deltaX:]):
    if((j-i)>stop): stop = idx
print(stop)
'''

fig = plt.figure()
ax = plt.gca()
ax.scatter(stop_words_list[:,0] ,stop_words_list[:,1] , s=1,c='blue', marker='.')
ax.set_yscale('log')
#ax.set_xscale('log')
fig.set_size_inches(10, 7)
plt.axvline(x=elbow_idx, color='r', linestyle='--')


relev_exp_unigrams = list_of_counts[:elbow_idx]
stop_words_unigrams = list_of_counts[elbow_idx:]